In [1]:
import numpy as np
from bs4 import BeautifulSoup
import pickle

#data = open("/home/user/Downloads/UKWAC-1_new.xml",'r', encoding='latin5').read()
#bs_data = BeautifulSoup(data, 'xml')
#del data
#pickle.dump(bs_data, open("UKWAC_1_BS4.pickle", "wb"))
bs_data = pickle.load(open("UKWAC_1_BS4.pickle",'rb'))

In [2]:
import nltk
from nltk.corpus import stopwords

# Create a set of stop words 
stop_words = set(stopwords.words('english')) 
stop_words = stop_words.union({".",",",":",";","?","!","#","\"","\'","(",")","@card@","I","-","\'s","n\'t","","...","[","]","@ord@","There","|","To","/",""})

In [4]:
from tqdm import tqdm
key_words_to_invert = ['The', 'Introduction', 'Alcohol', 'Asking', 'Experience', 'Comparison', 'Reported', 'Prevalence', 'Types', 'Trends', 'Weekly', 'Estimating', 'Frequency', 'Level', '', 'Daily', 'Amounts', 'Variations', 'Males', 'Proportion', '1998', 'by', 'Models', 'A', 'Efficient', 'Lighter', 'Higher', 'Processing', 'Work', 'Additional', 'Gaining', 'Improving', 'improving', 'TWI', 'Engaging', 'Fundamental', 'Aberdeen', 'For', '3/4', 'University', 'Brighton', '4-year', 'CCC', 'Cambridge', 'Sparkling', '3', 'Well', 'Cardiff', 'Mix', 'Sheffield', 'Computer', 'ABB', 'This', 'Governments', 'Too', 'We', 'When', 'Over', 'To', 'Tony', 'Carol', 'Ros', 'Peter', 'Janet', 'Richard', 'John', 'Patricia', 'There', 'Access', 'Public', 'SMEs', 'As', 'Large', 'Their', 'Separate', 'Also', 'Starting', 'Longer', 'Reforms', 'Many', 'Proportionately', 'Several', 'An', 'Outputs', 'At', 'Class', 'Sixth-form', 'It', 'Most', 'All', 'Regular', 'Almost', 'Students', 'Senior', 'In', 'Teachers', 'Although', 'Area-wide', 'Much', 'Colleges', 'Five', 'During', 'Basic', 'National', 'Support', 'Even', 'Youth', 'NVYOs', 'Standards', 'DfES', '35.1', 'Physical', 'Improved', 'Advances', 'Commercial', '1', 'Goddard', 'Todd', 'Smith', 'Gill', 'Human', 'Her', 'Further', 'Kessel', 'Source', 'Notes', 'Being', 'Ensuring', 'Identifying', 'radiation', 'occupational', 'liaison', '2001/02', 'Outturn', 'Overall', 'Building', 'Defence', 'Following', 'MOD', 'Emphasis', 'INDUSTRIAL', '(A)', 'INDUSTRY', 'Analysis', 'Well-established', 'Good', 'Easy', 'Lack', 'Small', 'End', 'Markets', 'Decline', 'Rapid', 'Strategic', 'Focus', 'Development', 'Modelling', 'Recommendations', 'limit', 'stop', 'an', 'Part', 'any']
unwanted_pos_tags = ['$', "''", '(', ')', ':','IN','LS','MD','PP','TO','VBD','VBN','WDT','WP$','WRB','``']
texts = bs_data.find_all("text")

for i in range(len(texts)):
    texts[i] = texts[i].find_all("s")
    
for i in tqdm(range(len(texts))):
    for j in range(len(texts[i])):
        temp_text = texts[i][j].text.split("\n")[1:-1]
        texts[i][j] = [[], []]
        for k in range(len(temp_text)):
            temp_text[k] = temp_text[k].split("\t")
            """
            The part from here is to put back in 
            place some POS tags in their right place
            """
            if len(temp_text[k])!=3:
                temp_text[k] = [temp_text[k][1], temp_text[k][2], temp_text[k][4]]
                #print(temp_text[k])
            
            current_pos = temp_text[k][1]
            
            if current_pos in key_words_to_invert:
                temp_text[k][1] = temp_text[k][2]
                temp_text[k][2] = current_pos
            
            """
            This part is to remove the stop words and punctuation from the set
            """
            if (temp_text[k][2].lower() not in stop_words) and (temp_text[k][1] not in unwanted_pos_tags):
                #texts[i][j][0].append(temp_text[k][0])
                texts[i][j][0].append(temp_text[k][1])
                texts[i][j][1].append(temp_text[k][2])
                

pickle.dump(texts, open("UKWAC_1_BS4_text_data_cleaned.pickle","wb"))

    

100%|██████████| 110165/110165 [03:52<00:00, 473.68it/s]


In [5]:
del bs_data

In [1]:
import pickle
from tqdm import tqdm
texts = pickle.load(open("/home/user/Documents/KULeuven/Master Thesis/lexsubcon-mod-test/baseline_bag_of_words/UKWAC_1_BS4_text_data_cleaned.pickle",'rb'))


In [6]:
"""

Vocabulary obtaining + POS conversion

"""
vocabulary = {}
vocabulary_pos = {}
for i_text in tqdm(range(len(texts))):
    for j_sentence in range(len(texts[i_text])):
        for k_word in range(len(texts[i_text][j_sentence][1])):
            if len(texts[i_text][j_sentence]) != 2:
                print("PROBLEM in data format")
                exit(1)
            current_word = texts[i_text][j_sentence][1][k_word]
            current_pos = texts[i_text][j_sentence][0][k_word]
            current_word = current_word
            if current_word not in vocabulary:
                vocabulary[current_word] = 1
            else:
                vocabulary[current_word] += 1
                
            if current_pos not in vocabulary_pos:
                vocabulary_pos[current_pos] = 1
            else:
                vocabulary_pos[current_pos] += 1

  0%|          | 0/110165 [00:00<?, ?it/s]

100%|██████████| 110165/110165 [00:46<00:00, 2379.00it/s]


In [7]:
import numpy as np
"""

Cell for Vocabulary Vizualization

"""
print(f"Size of vocabulary : {len(vocabulary)} (excluding stop words)")
counts = list(vocabulary.values())
index_counts_sorted = np.flip(np.argsort(counts))
words = list(vocabulary.keys())
print("\n----- Most frequent words in the vocabulary -----\n")
for i in range(10):
    index = int(index_counts_sorted[i])
    print(f"{i+1}\t\"\"{words[index]}\"\" : {counts[index]}")

print("\n\n--------------------------------------------------------\n\n")

print(f"Size of POS vocabulary : {len(vocabulary_pos)}")
counts_pos = list(vocabulary_pos.values())
index_counts_pos_sorted = np.flip(np.argsort(counts_pos))
pos = list(vocabulary_pos.keys())
print(f"All possible POS list : {pos} \n\n")
print("\n----- Most frequent POS in the vocabulary -----\n")
for i in range(10):
    index = int(index_counts_pos_sorted[i])
    print(f"{i+1}\t\"\"{pos[index]}\"\" : {counts_pos[index]}")

Size of vocabulary : 1181702 (excluding stop words)

----- Most frequent words in the vocabulary -----

1	""use"" : 192885
2	""make"" : 169935
3	""work"" : 156543
4	""one"" : 149868
5	""also"" : 143105
6	""year"" : 143103
7	""time"" : 135197
8	""take"" : 128222
9	""good"" : 121262
10	""new"" : 116554


--------------------------------------------------------


Size of POS vocabulary : 25
All possible POS list : ['NNS', 'JJ', 'NN', 'NP', 'VVD', 'VVP', 'CD', 'RB', 'VVN', 'VVZ', 'PDT', 'VVG', 'VV', 'JJS', 'DT', 'JJR', 'RBS', 'RBR', 'RP', 'CC', 'NPS', 'SYM', 'FW', 'UH', 'WP'] 



----- Most frequent POS in the vocabulary -----

1	""NN"" : 13156275
2	""NP"" : 9485255
3	""JJ"" : 5590429
4	""NNS"" : 5007844
5	""VV"" : 2402351
6	""VVN"" : 1974174
7	""RB"" : 1835151
8	""VVG"" : 1432667
9	""VVD"" : 943073
10	""VVZ"" : 748623
